In [1]:
# !pip install textblob
# !pip install spacy
# !python -m spacy download en_core_web_lg

In [2]:
import json
import os
import pandas as pd
import ast
import time
from sqlalchemy import create_engine
import psycopg2
from textblob import TextBlob
pd.options.display.max_columns = None

import statsmodels.api as sm
import numpy as np

/home/ubuntu/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
df_user=pd.read_csv("user.csv")

In [4]:
df_review=pd.read_csv("review_sentiment.csv")

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_user.head(1)

,Unnamed: 0,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,None,0,None,0,Susan,1,0,lzlZwIpuSWXEnNS91wxjHw,2015-09-28


In [6]:
df_review =df_review.merge(df_user,on="user_id")

In [7]:
df_review.head(1)

,Unnamed: 0_x,Unnamed: 0_x,business_id,cool_x,date,funny_x,review_id,stars_x,text,useful_x,user_id,Unnamed: 0_y,address,attributes,categories,city,hours,is_open,latitude,longitude,name_x,neighborhood,postal_code,review_count_x,stars_y,state,sentiment,Unnamed: 0_y,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool_y,elite,fans,friends,funny_y,name_y,review_count_y,useful_y,yelping_since
0,0,0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0.0,x7mDIiDB3jEiPGPHOmDzyw,2.0,The pizza was okay. Not the best I've had. I p...,0.0,msQe1u7Z_XuqjGoqhB0J5g,180795,3708 Las Vegas Blvd S,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...","Pizza, Restaurants",Las Vegas,"{'Monday': '11:0-5:0', 'Tuesday': '11:0-4:0', ...",1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV,0.416667,12988,3.31,2,0,2,1,0,0,0,0,0,0,0,1,None,1,"m_93Qy3Yhz31khEQVzqmqA, fNye_2mxMdQt7c4TFQdzww...",1,Melissa,11,4,2011-02-24


In [8]:
df_review = df_review.rename(index = str, columns = {"stars_x":"review_stars","stars_y":"average_business_stars"})

In [10]:
df_review = df_review.dropna().reset_index()

In [20]:
df_review.head(1)

,index,Unnamed: 0_x,Unnamed: 0_x,business_id,cool_x,date,funny_x,review_id,review_stars,text,useful_x,user_id,Unnamed: 0_y,address,attributes,categories,city,hours,is_open,latitude,longitude,name_x,neighborhood,postal_code,review_count_x,average_business_stars,state,sentiment,Unnamed: 0_y,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool_y,elite,fans,friends,funny_y,name_y,review_count_y,useful_y,yelping_since
0,0,0,0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0.0,x7mDIiDB3jEiPGPHOmDzyw,2.0,The pizza was okay. Not the best I've had. I p...,0.0,msQe1u7Z_XuqjGoqhB0J5g,180795,3708 Las Vegas Blvd S,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...","Pizza, Restaurants",Las Vegas,"{'Monday': '11:0-5:0', 'Tuesday': '11:0-4:0', ...",1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV,0.416667,12988,3.31,2,0,2,1,0,0,0,0,0,0,0,1,None,1,"m_93Qy3Yhz31khEQVzqmqA, fNye_2mxMdQt7c4TFQdzww...",1,Melissa,11,4,2011-02-24


zdgb

In [11]:
df_independent = df_review[['average_stars','average_business_stars','compliment_cool','compliment_plain','compliment_writer','review_count_y','fans','useful_y','funny_y','sentiment']]

In [12]:
#df_independent = df_review[['sentiment']]

In [13]:
df_dependent = df_review['review_stars']

In [14]:
X = sm.add_constant(df_independent)

In [15]:
model = sm.OLS(df_dependent,X).fit()

In [16]:
predictions = model.predict(X)

In [27]:
predictions[100]

4.058165589425762

In [28]:
df_review.loc[100,"review_stars"]

4.0

In [29]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           review_stars   R-squared:                       0.523
Model:                            OLS   Adj. R-squared:                  0.523
Method:                 Least Squares   F-statistic:                 2.814e+05
Date:                Tue, 14 Aug 2018   Prob (F-statistic):               0.00
Time:                        03:52:13   Log-Likelihood:            -3.5009e+06
No. Observations:             2563824   AIC:                         7.002e+06
Df Residuals:                 2563813   BIC:                         7.002e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -1.0368      0.004   -238.709      0.000      -1.045      -1.028
average_stars              0.5766      0.001    659.363      0.000       0.575       0.578
average_business_stars     0.5298      0.001    545.001      0.000       0.528       0.532
compliment_cool         2.431e-07   6.34e-06      0.038      0.969   -1.22e-05    1.27e-05
compliment_plain       -7.186e-06   4.18e-06     -1.718      0.086   -1.54e-05    1.01e-06
compliment_writer      -4.255e-06   1.32e-05     -0.322      0.747   -3.01e-05    2.16e-05
review_count_y          6.003e-05   2.24e-06     26.786      0.000    5.56e-05    6.44e-05
fans                    7.295e-05   1.78e-05      4.096      0.000     3.8e-05       0.000
useful_y                4.341e-06   4.68e-07      9.272      0.000    3.42e-06    5.26e-06
funny_y                -3.246e-06   6.13e-07     -5.293      0.000   -4.45e-06   -2.04e-06
sentiment                  2.5787      0.003    893.994      0.000       2.573       2.584
==============================================================================
Omnibus:                    57691.142   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            61783.546
Skew:                          -0.371   Prob(JB):                         0.00
Kurtosis:                       3.168   Cond. No.                     2.70e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.7e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""